In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.path.join(sys.path[0], '../src'))
import data
import utils

In [2]:
DATA_DIR = "../data/aksharantar_sampled/tam/"
os.chdir(DATA_DIR)

In [3]:
os.listdir()

['tam_test.csv', 'tam_train.csv', 'tam_valid.csv']

In [4]:
dataset = data.WordDataset()

In [8]:
dataset.get_onehot_encoding_x()

100%|██████████| 51200/51200 [00:41<00:00, 1243.38it/s]


In [10]:
ew = dataset.x_onehot[0]

In [11]:
dataset.decode_eng_word(ew)

'thottacharya'

In [12]:
ew[0].shape

torch.Size([128])

In [13]:
lf = nn.Linear(128, 16)
lf(ew[0]).shape

torch.Size([16])